In [ ]:
from  google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
import pandas as pd
import ast

In [ ]:
def open_file(file_path):

    df = pd.read_csv(file_path, delimiter='\t')
    return df

a = open_file('/content/drive/MyDrive/testa_di_Marianna/dati_wiki_ristretti/Marianna_head/wiki_naples_expanded_hyper.tsv')

def dictionary_formatter(data):
        """
        Converte un dataframe in una lista di dizionari nel formato {'title': title, 'text': text}.

        :param data: DataFrame di input.
        :return: Lista di dizionari formattati.
        """
        formatted_data = []

        for title, content in zip(data['title'], data['content']):
            try:
                content_dict = ast.literal_eval(content)
                for key, value in content_dict.items():
                    if key not in ['Voci correlate', 'Collegamenti esterni','Altri progetti','Note', 'Bibliografia']:
                        if isinstance(value, str):
                            formatted_data.append({'title': f"{title} - {key.lower()}", 'text': ' '.join(value.split('\n'))})
                        elif isinstance(value, dict):
                            for sub_key, sub_value in value.items():
                                if sub_key not in ['Voci correlate', 'Collegamenti esterni','Altri progetti','Note', 'Bibliografia']:
                                    formatted_data.append({'title': f"{title} - {sub_key.lower()}", 'text': ' '.join(sub_value.split('\n'))})
            except (ValueError, SyntaxError) as e:
                print(f"Errore nella conversione del contenuto per il titolo '{title}': {e}")

        return formatted_data

all_dicts = dictionary_formatter(a)

all_dicts_clean = [el for el in all_dicts if el['text'] != '']

all_par = [el['text'] for el in all_dicts_clean]
all_titles = [el['title'] for el in all_dicts_clean]

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
!pip install transformers accelerate huggingface-hub

In [ ]:
from huggingface_hub import login
login(token='REPLACE_WITH_YOUR_ACCESS_TOKEN')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
SYSTEM_PROMPT = """Sei un potenziale visitatore di una mostra dedicata alla storia e ai reperti della città di Napoli.
                   Il tuo compito è preparare delle domande che vorresti fare alla guida per sapere di più su ciò leggi.
                   Le domande dovrebbero essere semplici ma diversificate in natura del contenuto del documento.
                   Limita le domande al contesto delle informazioni fornite e non superare le 10 parole per domanda.
                   Formula le domande in lingua italiana in modo semplice e conciso."""

INSTRUCTION_PROMPT = """Ti viene presentato un testo scritto riguardante storia o archeologia di Napoli o località correlate.
                        Il tuo compito è formulare domande dirette pertinenti al paragrafo in modo che rispondano al paragrafo stesso.
                        Le domande devono essere formulate partendo da "Chi", "Cosa", "Dove", "Quando", "Come","Perchè", "Quale". Considera anche formulazioni composte e passive.
                        Segui gli esempi per stile e struttura quando formuli le tue domande. Non fare introduzioni.
                        Considera la stringa {title} come argomento principale del testo e rendila SEMPRE esplicita all'interno della domanda.
                        La tue domande deve essere rispondibili dal testo, ma non copiare il testo parola per parola. Puoi usare anche i sinonimi per generare più domande.
                        Dammi come output SOLO le domande. Crea più domande possibili, ma fermati se non elemeni a disposizione
                        Formatta l'output in una lista di stringhe come mostrato negli esempi che seguono.

                        Esempi:

                        1. paragrafo: Nella Grecia antica indicava quella parte della città che veniva costruita per ragioni difensive sulla sommità di un\'altura e spesso cinta da mura.
                        Iniziata a diffondersi nell\'età del bronzo, la ""parte alta"" delle città greche, come Atene, Pergamo, Argo, Micene e Tirinto, in età micenea era il luogo di residenza del re, ma col tempo divenne il centro religioso dell\'abitato, sede di templi e luoghi di riunione.
                        Essa si contrapponeva alla zona denominata asty, la parte periferica in cui viveva il popolo. Al suo interno c\'erano case, templi e la piazza principale, l\'agorà.
                        Esempi tipici di acropoli, alcune in parte tuttora ben conservate, sono quelli di Atene, Corinto,Selinunte (Sicilia), di Canosa di Puglia e di Taranto (Taras);
                        anche il castello Eurialo di Siracusa può essere considerato parte dell\'acropoli di quella città. L\'acropoli di Atene, una collina rocciosa che si eleva fino a circa 100 metri sulla città circostante, resta l\'esempio più noto al mondo.
                        Tali fortificazioni non sono riferibili esclusivamente al mondo ellenico, bensì si ritrovano in tutto il Mediterraneo orientale (ad esempio presso gli Ittiti) e, dall\'età del ferro, in Italia (ad esempio l\'acropoli di Alatri, Ferentino, Cuma e Arpino sulle origini delle quali, tra l\'altro, non è esclusa la matrice mediorientale). Recenti ricerche retrodaterebbero di molto la fondazione dell\'Acropoli di Alatri.\nIn età romana, la funzione monumentale si sostituì a quella difensiva, che tuttavia venne recuperata nell\'alto Medioevo.

                        output: ['Come si descrive l'acropoli nell'antica Grecia?', 'Cosa indicava l'acropoli nell'antica Grecia?', 'Dove viveva il popolo fuori dall'acropoli?','Quali sono le acropoli più famose?']


                        2. paragrafo: Qualche precedente storico si può far risalire alle piazze minoiche, a Creta, luogo nel quale sono stati rintracciati le prime agorai.
                        Ad Argo il primitivo impianto dell’agorà era ai piedi della Larissa, sui resti di un piccolo abitato geometrico affiancato da una necropoli, ""la cui datazione sembra risalire al pieno VI sec. a.C."":
                        in effetti, gli agorai arcaici furono strettamente associati a santuari religiosi e ad attività di intrattenimento, quali feste, giochi e teatro.
                        Con l\'andare del tempo l\'agorà divenne il centro della polis sia dal punto di vista economico e commerciale (in quanto sede del mercato) che dal punto di vista religioso, poiché vi si trovavano i luoghi di culto del fondatore della città o della divinità protettrice, oltreché politico, in quanto era il luogo della democrazia per antonomasia, dato che era sede delle assemblee dei cittadini che vi si riunivano per discutere i problemi della comunità e decidere collegialmente sulle leggi, ma era contemporaneamente il luogo del mercato e il centro economico e politico, e perciò vi sorgevano gli edifici pubblici, gli uffici, i teatri.
                        L\'agorà fu un\'autentica invenzione urbanistica, che non trovò riscontro né nei centri del vicino Oriente né in quelli micenei.\nQuesta innovazione fu inserita grazie alle grandi modifiche urbanistiche iniziate durante l\'età di Pericle intorno al V secolo a.C., che con il tempo, durante l\'età ellenistica portarono a tre tipologie principali di agorà: quella mercantile, nelle città marinare, in stretta connessioni ai porti, quella commerciale, per la quale l\'agorà era situata presso le porte della città, e quella politico-religiosa che prevedeva una collocazione nel centro della città.
                        Nello stesso periodo storico, le agorà vennero delimitate da portici (stoài) e si appoggiavano spesso ad una via di transito principale. Non mancarono esempi di agorai con terrazzamenti e con portici situati su più piani. Durante l\'ellenismo venne svolto un lavoro di riordino dei vari monumenti, costruzioni, are, statue, che nei periodi precedenti avevano finito per collocarsi disordinatamente nella piazza, anche se i greci, diversamente dagli antichi romani, non perseguirono l\'assialità, la frontalità e la simmetria dal punto di vista urbanistico e quindi le agorà non furono mai subordinate ad una costruzione principale.'

                        output: ['Qual è l'origine dell'agorà?', 'Dove si trovava una primitiva agorà?', 'Quali erano le funzioni dell'agorà?, 'A quando risale la prima testimonianza di agorà?']

                        3. paragrafo: Nell'agorà, dunque, si mantenevano o si creavano numerose relazioni interpersonali e vi si prendevano numerose decisioni, da cui però venivano escluse le donne.
                        Essendo il cuore pulsante di ogni attività era situata nella città, la città bassa; la sua funzione politica venne acquisita quando, terminata l'epoca micenea che vedeva un re al comando, le istituzioni furono spostate appunto nella città bassa, mentre precedentemente si trovavano nel palazzo reale situato sull'acropoli.
                        Nella polis tutti coloro che possedevano la qualifica di cittadini avevano gli stessi diritti e gli stessi doveri: si riunivano in assemblea ed eleggevano i magistrati, cioè gli esecutori del volere collettivo. Le poleis erano principalmente delle città-Stato. Famosi filosofi greci dicevano che la polis di Atene era un luogo con case, mercati, templi e teatri, ma che erano gli ateniesi a fare la polis. Dopo la Guerra del Peloponneso l'agorà e l'acropoli erano collegate da una lunga cinta muraria che le difendeva da attacchi esterni.

                        output: ['Quali erano le funzioni dell'agorà?, 'Dove furono spostate le istituzioni dell'agorà?', 'Chi poteva partecipare nell'agorà?']

                        """

PROMPT = """Analizza il seguente paragrafo e formula domande semplici, dirette e pertinenti non superiori alle 10 parole che possono essere risposte leggendo attentamente il paragrafo.
            Riporta SOLAMENTE l'output senza introduzioni e numerazioni.

            paragrafo: {context}

            output: """

In [ ]:
def quest_gen(tit, par):

  messages = [
      {"role": "system", "content": SYSTEM_PROMPT},
      {"role": "user", "content": INSTRUCTION_PROMPT.format(title=tit)},
      {"role": "user", "content": PROMPT.format(context=par)}
]

  outputs = pipe(
      messages,
      max_new_tokens=250,
      num_return_sequences=1,
      pad_token_id=tokenizer.eos_token_id
  )

  return outputs[0]["generated_text"][-1]['content']

In [ ]:
zipped = list(zip(all_titles[3:], all_par[3:]))

In [ ]:
q = [[quest_gen(el[0], el[-1])] for el in zipped]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
all_q = [['Come si descrive l\'acropoli \'antica Grecia?', 'Cosa indicava l\'acropoli nell\'antica Grecia?', 'Dove viveva il popolo fuori dall\'acropoli?','Quali sono le acropoli più famose?'],
        ['Qual è l\'origine dell\'agorà?', 'Dove si trovava una primitiva agorà?', 'Quali erano le funzioni dell\'agorà?', 'A quando risale la prima testimonianza di agorà?'],
        ['Quali erano le funzioni dell\'agorà?', 'Dove furono spostate le istituzioni dell\'agorà?', 'Chi poteva partecipare nell\'agorà?']] + q

In [ ]:
dataq = {'titles': all_titles,
        'text': all_par,
        'question': all_q}


dfq = pd.DataFrame(dataq)
dfq.to_csv('/content/drive/MyDrive/testa_di_Marianna/dati_wiki_ristretti/wiki_naples_wquest.tsv', sep='\t', index=False)